# Explanations
* Attaches a column to the recommendations about why the item was recommended

In [ ]:
import JSON

In [ ]:
function get_utility_coefs(medium, task, alpha)
    prev_log_level = Logging.min_enabled_level(Logging.global_logger())
    Logging.disable_logging(Logging.Warn)
    medium = "anime"
    task = "temporal_causal"
    m = read_params("$medium/$task/$alpha")["m"]
    Logging.disable_logging(prev_log_level)
    m = getfield(m, :fields)[:layers][1].paths
    vcat(vec(m[1].W), vec(m[2].W))
end

In [ ]:
function load_attributions(username, medium, task, alpha, items)
    coefs = get_utility_coefs(medium, task, alpha)
    cmd = [
        "python3",
        "IntegratedGradients.py",
        "--username",
        "$username",
        "--medium",
        "$medium",
        "--task",
        "$task",
        "--coefs",
        string.(coefs)...,
        "--items",
        string.(items)...,
    ]
    run(`$cmd`)
    cache_fn = (
        "../../data/recommendations/$username/" *
        "explanations/$medium/integrated_gradients.json"
    )
    f = open(cache_fn)
    attrs = JSON.parse(f)
    close(f)
    attrs
end;

In [ ]:
function get_explanation(attrs, item, topn)
    influences = attrs["attributions"][string(item)]
    sorted_attrs = sortperm(influences)
    explanations = []
    for n = 1:topn
        idx = sorted_attrs[end+1-n]
        if influences[idx] == 0
            continue
        end
        for m in ["anime", "manga"]
            uid = attrs[m][idx]
            if uid < num_items(m)
                push!(explanations, get_link(m, source, uid))
            end
        end
    end
    explanations
end;

In [ ]:
function attach_explanations!(rec_df, medium, task, alpha, N)
    items = rec_df.uid .- 1
    attrs = load_attributions(username, medium, task, alpha, items)
    expls = [get_explanation(attrs, x, N) for x in items]
    formatted_expls = [join(string.(x), ", ") for x in expls]
    col = :because_you_liked
    rec_df[:, col] = formatted_expls
    rec_df
end;